In [2]:
%run ./read_file.ipynb

In [3]:
from graphframes import GraphFrame

In [ ]:
# df = readCheckpoint(size='medium').cache()

In [ ]:
# df.printSchema()

In [ ]:
#filter to 3 malicious hosts

In [ ]:
# df_first_events = get_firsts(df).cache()

In [ ]:
# df_first_events.count()

In [ ]:
# x is the number of hours
# y is the maximum number of events for any hour in a graph
# x_axis = 2*(x-1)+2
# y_axis = 2*(y-1)+2

In [ ]:
# df_first_events.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/first_events")

In [4]:
df = spark.read.parquet(f"{s3_url_trusted}/prod/graph/first_events")\
            .withColumn("image_path", getFileUDF(col("image_path"))) \
            .withColumn("parent_image_path", getFileUDF(col("parent_image_path"))) \
            .withColumn("new_path", getFileUDF(col("new_path"))) \
            .withColumn("file_path", getFileUDF(col("file_path"))) \
            .cache()

In [4]:
df.limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action,hostname,user_name,privileges,image_path,...,file_path,direction,logon_id,requesting_domain,requesting_user,event_minute,event_day,event_hour,malicious,relationship
0,4cc78dc0-6326-4ec2-ba10-45e3de3a9fc9,2019-09-23 17:10:12.127,00008f39-ef23-4bbe-b016-6f38be0eb59e,22e33a78-6ee0-418c-89ee-51384377ac90,FILE,READ,SysClient0465.systemia.com,None,None,firefox.exe,...,fastprox.dll,None,None,None,None,10,23,17,0,00008f39-ef23-4bbe-b016-6f38be0eb59e->22e33a78...
1,3256faf6-8ca4-4229-b5b7-7891a326acad,2019-09-23 17:52:46.260,0000909a-c9bf-4842-a2a3-0d0b37c5164d,9699bec5-89d2-4982-8d45-b1e0bf318ed9,FILE,READ,SysClient0119.systemia.com,None,None,WScript.exe,...,desktop.ini,None,None,None,None,52,23,17,0,0000909a-c9bf-4842-a2a3-0d0b37c5164d->9699bec5...
2,b3ba2302-277a-4788-8216-96a51c276c56,2019-09-23 16:27:02.217,0000ec57-02e1-4a13-99c7-2a617fb8ac95,7eb5c010-7e92-4896-ba2f-6bc58f716626,FILE,READ,SysClient0667.systemia.com,None,None,conhost.exe,...,easy-install.pth,None,None,None,None,27,23,16,0,0000ec57-02e1-4a13-99c7-2a617fb8ac95->7eb5c010...
3,48964880-a761-4125-a6d5-e9b8a4f21009,2019-09-23 15:46:43.448,00018ce5-0c98-4366-b0e1-1739a7f816c7,0fb964c3-3809-42b3-b219-abb3e5e0060f,FLOW,INFO,SysClient0775.systemia.com,None,None,None,...,None,None,None,None,None,46,23,15,0,00018ce5-0c98-4366-b0e1-1739a7f816c7->0fb964c3...
4,4edf49e8-0cd7-4165-b0dd-ad15c7c5bad3,2019-09-23 19:12:03.396,0001cab7-a0ff-4873-a421-01c3dc18aa20,40d68792-8340-4ed9-b354-3992d19fe1ea,FILE,READ,SysClient0952.systemia.com,None,None,python.exe,...,action_runners,None,None,None,None,12,23,19,0,0001cab7-a0ff-4873-a421-01c3dc18aa20->40d68792...


In [5]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- relationship: string (nullable = true)



In [8]:
df.count()

31170587

In [7]:
print("image path: "+str(df.select("image_path").distinct().count()))
print("image path: "+str(df.select("parent_image_path").distinct().count()))

image path: 153
image path: 151


In [9]:
df = df.limit(1000).cache()

In [10]:
df.count()

1000

In [11]:
# Create distinct vertices with source as actorid, destination as objectid
src_vertices = df.selectExpr('actorID as id').distinct()
dst_vertices = df.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [ ]:
# display the vertices
print("Vertices:")
g.vertices.show()

In [ ]:
# display the edges
print("Edges:")
g.edges.show(5)

In [ ]:
g.inDegrees.show(5)

In [21]:
# find all paths of length 6 in the graph
# Handling null values in shorter length 
motifs2 = g.find("(a)-[e1]->(b); (b)-[e2]->(c)")
#motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [15]:
# filter paths to only those where all edges are connected
connected_paths = motifs2.filter("e1.dst = e2.src")# and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src")



In [22]:
# filter paths to only those where all edges are connected
connected_paths = motifs6.filter("e1.dst = e2.src and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src")


In [ ]:
# get the 6 edges in each path and convert to a pandas dataframe
connected_paths.select("e1").limit(1).collect()

In [11]:
type(connected_paths)

pyspark.sql.dataframe.DataFrame

In [23]:
connected_paths.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test")

In [24]:
connected_paths = spark.read.parquet(f"{s3_url_trusted}/prod/graph/test").cache()

In [25]:
connected_paths.limit(5).toPandas()

,a,e1,b,e2,c,e3,d,e4,e,e5,f,e6,g
0,"(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)"
1,"(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25...","(0006e259-0f61-4bbd-b9dc-9c4a7f30004e,)"
2,"(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)"
3,"(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)"
4,"(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)","(000ab363-f760-47f6-a369-60c14ac9dcde, 000ab36...","(000ab363-f760-47f6-a369-60c14ac9dcde,)"


In [26]:
connected_paths.count()

273

In [39]:
#transpose values first by creating a trace column
# Add a new column "new_col" with ascending values starting at 1 before all other columns
df_transp = connected_paths.withColumn("Trace", (monotonically_increasing_id() + 1))
df_transp = df_transp.select("Trace", 
                             *[col for col in df_transp.columns if col != "Trace"])


In [40]:
df_transp.limit(1).toPandas()

,Trace,a,e1,b,e2,c,e3,d,e4,e,e5,f,e6,g
0,1,"(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)","(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73...","(0005a73d-e2af-4e57-b432-d8cd419ae274,)"


In [41]:
#drop all vertices
df_transp = df_transp.drop('a','b','c','d','e','f','g')

In [42]:
df_transp.printSchema()

root
 |-- Trace: long (nullable = false)
 |-- e1: struct (nullable = true)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |    |-- timestamp: timestamp (nullable = true)
 |    |-- object: string (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- hostname: string (nullable = true)
 |    |-- user_name: string (nullable = true)
 |    |-- privileges: string (nullable = true)
 |    |-- image_path: string (nullable = true)
 |    |-- parent_image_path: string (nullable = true)
 |    |-- new_path: string (nullable = true)
 |    |-- file_path: string (nullable = true)
 |    |-- direction: string (nullable = true)
 |    |-- logon_id: string (nullable = true)
 |    |-- requesting_domain: string (nullable = true)
 |    |-- requesting_user: string (nullable = true)
 |    |-- malicious: integer (nullable = true)
 |-- e2: struct (nullable = true)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |    |-- timestamp: t

In [50]:
#transpose rows 
stacked_df = df_transp.selectExpr(
    "Trace", 
    "posexplode(array(e1, e2, e3, e4, e5, e6)) as (pos, col)"
).select(
    "Trace", 
    expr('''CASE pos 
    WHEN 0 THEN 'e1' 
    WHEN 1 THEN 'e2'
    WHEN 2 THEN 'e3'
    WHEN 3 THEN 'e4'
    WHEN 4 THEN 'e5'
    ELSE 'e6' END''').alias("event"),
    "col"
).orderBy("Trace","event")

stacked_df.limit(7).toPandas()

,Trace,event,col
0,1,e1,"(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73..."
1,1,e2,"(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73..."
2,1,e3,"(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73..."
3,1,e4,"(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73..."
4,1,e5,"(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73..."
5,1,e6,"(0005a73d-e2af-4e57-b432-d8cd419ae274, 0005a73..."
6,2,e1,"(0006e259-0f61-4bbd-b9dc-9c4a7f30004e, 0006e25..."


In [51]:
#explode columns
df.select(*df.columns, "properties.*").drop('properties')

root
 |-- Trace: long (nullable = false)
 |-- event: string (nullable = false)
 |-- col: struct (nullable = true)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |    |-- timestamp: timestamp (nullable = true)
 |    |-- object: string (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- hostname: string (nullable = true)
 |    |-- user_name: string (nullable = true)
 |    |-- privileges: string (nullable = true)
 |    |-- image_path: string (nullable = true)
 |    |-- parent_image_path: string (nullable = true)
 |    |-- new_path: string (nullable = true)
 |    |-- file_path: string (nullable = true)
 |    |-- direction: string (nullable = true)
 |    |-- logon_id: string (nullable = true)
 |    |-- requesting_domain: string (nullable = true)
 |    |-- requesting_user: string (nullable = true)
 |    |-- malicious: integer (nullable = true)



In [18]:
# using split and applying in ml with pypark
train, test, val = connected_paths.randomSplit([0.80, 0.15, 0.05], seed=12345)

In [22]:
val.count()

9

In [ ]:
# create new dataframe to union and create duplicate rows
df_dup = df.withColumn('id', lit(None).cast(StringType()))

In [ ]:
df_dup.limit(5).toPandas()

In [ ]:
df_new = df.union(df_dup).sort("timestamp").cache()
df.unpersist()

In [ ]:
df_new.limit(5).toPandas()

In [ ]:
# len(df_new.select('relationship').distinct().collect())

In [ ]:
tot = df_new.count()

In [ ]:
df_ratios = df_new.groupBy("object") \
              .count() \
              .withColumnRenamed('count', 'cnt_per_group') \
              .withColumn('perc_of_count_total', (col('cnt_per_group') / tot) * 100 ) \
              .cache()

In [ ]:
df_ratios.limit(5).toPandas()

In [ ]:
df_new.limit(5).toPandas()

In [ ]:
connected_paths.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test")